In [11]:
import pandas as pd
import datetime as dt
import numpy as np


In [37]:
symbol = "JPM"
start_date = dt.datetime(2008, 1, 1)
end_date = dt.datetime(2009, 12, 31)
df = pd.read_csv("data/{}.csv".format(symbol), index_col="Date", parse_dates=True)
df.drop(["Close"], axis=1, inplace=True)
df = df.rename(columns={"Adj Close": "Close"})
df


,Open,High,Low,Volume,Close
Date,,,,,
2012-09-12,39.88,40.25,39.79,29882200,39.92
2012-09-11,38.80,39.63,38.73,22378900,39.60
2012-09-10,39.23,39.78,38.73,20853500,38.76
2012-09-07,39.04,39.53,39.00,28082400,39.30
2012-09-06,37.58,38.87,37.50,41346800,38.69
...,...,...,...,...,...
2000-02-07,82.19,83.37,81.75,4684200,37.37
2000-02-04,82.50,83.31,81.56,5115100,37.17
2000-02-03,83.50,84.87,80.25,9200200,37.82


In [40]:
def MACD(df: pd.DataFrame, short_window=12, long_window=26, signal_window=9)->pd.DataFrame:
    # Calculate the short / long term exponential moving average (short_window EMA)
    short_ema = df["Close"].ewm(span=short_window, adjust=False).mean()
    long_ema = df["Close"].ewm(span=long_window, adjust=False).mean()
    macd = short_ema - long_ema
    signal = macd.ewm(span=signal_window, adjust=False).mean()
    histogram = macd - signal
    buy_signal = (macd.shift(1) < signal.shift(1)) & (macd > signal)
    sell_signal = (macd.shift(1) > signal.shift(1)) & (macd < signal)
    df_macd = pd.DataFrame({'MACD': macd, 'Signal Line': signal, 'Histogram': histogram, 'Buy Signal': buy_signal, 'Sell Signal': sell_signal})

    return df_macd


MACD(df)



,MACD,Signal Line,Histogram,Buy Signal,Sell Signal
Date,,,,,
2012-09-12,0.000000,0.000000,0.000000,False,False
2012-09-11,-0.025527,-0.005105,-0.020422,False,False
2012-09-10,-0.112245,-0.026533,-0.085711,False,False
2012-09-07,-0.135830,-0.048392,-0.087437,False,False
2012-09-06,-0.201421,-0.078998,-0.122423,False,False
...,...,...,...,...,...
2000-02-07,-0.008501,-0.385539,0.377038,False,False
2000-02-04,0.005611,-0.307309,0.312920,False,False
2000-02-03,0.068455,-0.232156,0.300611,False,False


In [52]:

def BollingerBands(df: pd.DataFrame, window=20, num_std=2) -> pd.DataFrame:
    rolling_mean = df["Close"].rolling(window=window).mean()
    rolling_std = df["Close"].rolling(window=window).std()
    upper_band = rolling_mean + (rolling_std * num_std)
    lower_band = rolling_mean - (rolling_std * num_std)
    bbp = (df["Close"] - lower_band) / (upper_band - lower_band)

    # Create a DataFrame that contains the original price, rolling mean, upper band, lower band, and signals
    df_bollinger = pd.DataFrame({
        'Rolling Mean': rolling_mean,
        'Upper Band': upper_band,
        'Lower Band': lower_band,
        'Signal': bbp
    })

    return df_bollinger

BollingerBands(df)


,Rolling Mean,Upper Band,Lower Band,Signal
Date,,,,
2012-09-12,NaN,NaN,NaN,NaN
2012-09-11,NaN,NaN,NaN,NaN
2012-09-10,NaN,NaN,NaN,NaN
2012-09-07,NaN,NaN,NaN,NaN
2012-09-06,NaN,NaN,NaN,NaN
...,...,...,...,...
2000-02-07,36.365,38.980472,33.749528,0.692126
2000-02-04,36.327,38.867751,33.786249,0.665896
2000-02-03,36.374,38.991443,33.756557,0.776224


In [45]:
def RSI(df: pd.DataFrame, window=14)->pd.DataFrame:
    
    # Calculate daily returns
    daily_returns = df["Close"].diff()
    # Calculate up days and down days
    up_days = daily_returns.where(daily_returns > 0, 0)
    down_days = -daily_returns.where(daily_returns < 0, 0)
    # Calculate average gain and average loss
    avg_gain = up_days.rolling(window=window).mean()
    avg_loss = down_days.rolling(window=window).mean()
    # Calculate relative strength
    rs = avg_gain / (avg_loss + 1e-10)
    # Calculate RSI
    rsi = 100 - (100 / (1 + rs))
    
    
    rsi_df = pd.DataFrame({
        "Close_RSI": rsi
    })
    return rsi_df


RSI(df)


,Close_RSI
Date,
2012-09-12,NaN
2012-09-11,NaN
2012-09-10,NaN
2012-09-07,NaN
2012-09-06,NaN
...,...
2000-02-07,58.931860
2000-02-04,63.241107
2000-02-03,65.266106


In [55]:
# Function to compute volume-based features with lowercase column names
def compute_volume_features(df):
    df = df.copy()
    
    # 1. On-Balance Volume (OBV)
    df['OBV'] = (np.sign(df['Close'].diff()) * df['Volume']).fillna(0).cumsum()
    
    # 2. Volume Rate of Change (VROC)
    df['VROC_10'] = df['Volume'].pct_change(10)
    df['VROC_20'] = df['Volume'].pct_change(20)

    # 3. Accumulation/Distribution Index (ADI)
    df['ADI'] = ((df['Close'] - df['Low']) - (df['High'] - df['Close'])) / (df['High'] - df['Low']) * df['Volume']
    df['ADI_CUMSUM'] = df['ADI'].cumsum()

    # 4. Volume Weighted Average Price (VWAP)
    df['VWAP'] = (df['Volume'] * (df['High'] + df['Low'] + df['Close']) / 3).cumsum() / df['Volume'].cumsum()

    # 5. Moving Average of Volume (SMA of Volume)
    df['SMA_VOL_10'] = df['Volume'].rolling(window=10).mean()
    df['SMA_VOL_20'] = df['Volume'].rolling(window=20).mean()

    # 6. Exponential Moving Average of Volume (EMA of Volume)
    df['EMA_VOL_10'] = df['Volume'].ewm(span=10, adjust=False).mean()
    df['EMA_VOL_20'] = df['Volume'].ewm(span=20, adjust=False).mean()

    # 7. Volume Standard Deviation
    df['VOL_STD_10'] = df['Volume'].rolling(window=10).std()
    df['VOL_STD_20'] = df['Volume'].rolling(window=20).std()

    # 8. Volume Z-Score (Normalization)
    df['VOLUME_ZSCORE'] = (df['Volume'] - df['Volume'].rolling(window=20).mean()) / df['Volume'].rolling(window=20).std()

    # 9. Volume Trend Strength (Difference from Moving Average)
    df['VOLUME_TREND'] = df['Volume'] - df['SMA_VOL_10']

    # 10. Volume Surge Indicator (Ratio of Current Volume to 10-day Average)
    df['VOLUME_SURGE'] = df['Volume'] / df['SMA_VOL_10']

    return df


# Compute volume-based features with lowercase columns
compute_volume_features(df)




,Open,High,Low,Volume,Close,OBV,VROC_10,VROC_20,ADI,ADI_CUMSUM,VWAP,SMA_VOL_10,SMA_VOL_20,EMA_VOL_10,EMA_VOL_20,VOL_STD_10,VOL_STD_20,VOLUME_ZSCORE,VOLUME_TREND,VOLUME_SURGE
Date,,,,,,,,,,,,,,,,,,,,
2012-09-12,39.88,40.25,39.79,29882200,39.92,0.000000e+00,NaN,NaN,-1.299226e+07,-1.299226e+07,39.986667,NaN,NaN,2.988220e+07,2.988220e+07,NaN,NaN,NaN,NaN,NaN
2012-09-11,38.80,39.63,38.73,22378900,39.60,-2.237890e+07,NaN,NaN,2.088697e+07,7.894712e+06,39.701191,NaN,NaN,2.851796e+07,2.916760e+07,NaN,NaN,NaN,NaN,NaN
2012-09-10,39.23,39.78,38.73,20853500,38.76,-4.323240e+07,NaN,NaN,-1.966187e+07,-1.176716e+07,39.526869,NaN,NaN,2.712442e+07,2.837578e+07,NaN,NaN,NaN,NaN,NaN
2012-09-07,39.04,39.53,39.00,28082400,39.30,-1.515000e+07,NaN,NaN,3.708996e+06,-8.058163e+06,39.457437,NaN,NaN,2.729860e+07,2.834784e+07,NaN,NaN,NaN,NaN,NaN
2012-09-06,37.58,38.87,37.50,41346800,38.69,-5.649680e+07,NaN,NaN,3.048195e+07,2.242378e+07,39.137177,NaN,NaN,2.985282e+07,2.958584e+07,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2000-02-07,82.19,83.37,81.75,4684200,37.37,1.326891e+09,-0.337323,0.025685,-2.613321e+08,-5.312998e+11,36.692546,6829170.0,6099910.0,6.549147e+06,6.512176e+06,1.705381e+06,1.557889e+06,-0.908736,-2144970.0,0.685911
2000-02-04,82.50,83.31,81.56,5115100,37.17,1.321776e+09,-0.370162,-0.101258,-2.646114e+08,-5.315644e+11,36.694587,6528550.0,6071095.0,6.288411e+06,6.379121e+06,1.717221e+06,1.571116e+06,-0.608481,-1413450.0,0.783497
2000-02-03,83.50,84.87,80.25,9200200,37.82,1.330976e+09,0.609496,0.538135,-1.781892e+08,-5.317426e+11,36.698295,6876950.0,6232035.0,6.817827e+06,6.647796e+06,1.879823e+06,1.719316e+06,1.726363,2323250.0,1.337831


In [57]:
combined_df = pd.concat([compute_volume_features(df), MACD(df), BollingerBands(df), RSI(df)], axis=1)
combined_df


,Open,High,Low,Volume,Close,OBV,VROC_10,VROC_20,ADI,ADI_CUMSUM,...,MACD,Signal Line,Histogram,Buy Signal,Sell Signal,Rolling Mean,Upper Band,Lower Band,Signal,Close_RSI
Date,,,,,,,,,,,,,,,,,,,,,
2012-09-12,39.88,40.25,39.79,29882200,39.92,0.000000e+00,NaN,NaN,-1.299226e+07,-1.299226e+07,...,0.000000,0.000000,0.000000,False,False,NaN,NaN,NaN,NaN,NaN
2012-09-11,38.80,39.63,38.73,22378900,39.60,-2.237890e+07,NaN,NaN,2.088697e+07,7.894712e+06,...,-0.025527,-0.005105,-0.020422,False,False,NaN,NaN,NaN,NaN,NaN
2012-09-10,39.23,39.78,38.73,20853500,38.76,-4.323240e+07,NaN,NaN,-1.966187e+07,-1.176716e+07,...,-0.112245,-0.026533,-0.085711,False,False,NaN,NaN,NaN,NaN,NaN
2012-09-07,39.04,39.53,39.00,28082400,39.30,-1.515000e+07,NaN,NaN,3.708996e+06,-8.058163e+06,...,-0.135830,-0.048392,-0.087437,False,False,NaN,NaN,NaN,NaN,NaN
2012-09-06,37.58,38.87,37.50,41346800,38.69,-5.649680e+07,NaN,NaN,3.048195e+07,2.242378e+07,...,-0.201421,-0.078998,-0.122423,False,False,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2000-02-07,82.19,83.37,81.75,4684200,37.37,1.326891e+09,-0.337323,0.025685,-2.613321e+08,-5.312998e+11,...,-0.008501,-0.385539,0.377038,False,False,36.365,38.980472,33.749528,0.692126,58.931860
2000-02-04,82.50,83.31,81.56,5115100,37.17,1.321776e+09,-0.370162,-0.101258,-2.646114e+08,-5.315644e+11,...,0.005611,-0.307309,0.312920,False,False,36.327,38.867751,33.786249,0.665896,63.241107
2000-02-03,83.50,84.87,80.25,9200200,37.82,1.330976e+09,0.609496,0.538135,-1.781892e+08,-5.317426e+11,...,0.068455,-0.232156,0.300611,False,False,36.374,38.991443,33.756557,0.776224,65.266106


In [ ]:
combined_df.to_csv("featured_data/{}.csv".format(symbol))

## Automation

In [60]:
import os
# Create the output folder if it does not exist
os.makedirs("featured_data", exist_ok=True)

# Process all CSV files in the "data" directory
data_folder = "data"

for file in os.listdir(data_folder):
    if file.endswith(".csv"):
        symbol = file.replace(".csv", "")
        file_path = os.path.join(data_folder, file)

        # Load data
        try:
            df = pd.read_csv(file_path, index_col="Date", parse_dates=True)
        except Exception as e:
            print(f"Error reading {file_path}: {str(e)}")
            continue

        # Process DataFrame
        if "Close" in df.columns:
            df.drop(["Close"], axis=1, inplace=True)
        df = df.rename(columns={"Adj Close": "Close"})

        # Apply feature engineering
        combined_df = pd.concat([compute_volume_features(df), MACD(df), BollingerBands(df), RSI(df)],axis=1)

        # Save to new CSV
        output_file = os.path.join("featured_data", f"{symbol}.csv")
        combined_df.to_csv(output_file)
        print(f"Processed and saved: {output_file}")

Processed and saved: featured_data/PHM.csv
Processed and saved: featured_data/ML4T-309.csv
Processed and saved: featured_data/FRX.csv
Processed and saved: featured_data/GLD.csv
Processed and saved: featured_data/JPM.csv
Processed and saved: featured_data/ML4T-357.csv
Processed and saved: featured_data/HIG.csv
Processed and saved: featured_data/LEN.csv
Processed and saved: featured_data/BSC.csv
Processed and saved: featured_data/AVY.csv
Processed and saved: featured_data/ML4T-239.csv


/tmp/ipykernel_892722/2545850965.py:9: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['VROC_10'] = df['Volume'].pct_change(10)
/tmp/ipykernel_892722/2545850965.py:10: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['VROC_20'] = df['Volume'].pct_change(20)


Processed and saved: featured_data/ML4T-072.csv
Processed and saved: featured_data/THC.csv
Processed and saved: featured_data/ML4T-041.csv
Processed and saved: featured_data/ML4T-027.csv
Processed and saved: featured_data/ML4T-381.csv
Processed and saved: featured_data/IVZ.csv
Processed and saved: featured_data/ML4T-156.csv
Processed and saved: featured_data/BJS.csv
Processed and saved: featured_data/SINE_SLOW_NOISE.csv
Processed and saved: featured_data/ML4T-051.csv
Processed and saved: featured_data/SNV.csv
Processed and saved: featured_data/CVH.csv
Processed and saved: featured_data/ML4T-145.csv
Processed and saved: featured_data/FAKE2.csv
Processed and saved: featured_data/MMM.csv
Processed and saved: featured_data/ML4T-105.csv
Processed and saved: featured_data/ML4T-234.csv
Processed and saved: featured_data/ML4T-286.csv
Processed and saved: featured_data/ML4T-261.csv
Processed and saved: featured_data/XRAY.csv
Processed and saved: featured_data/ML4T-066.csv
Processed and saved: f

/tmp/ipykernel_892722/2545850965.py:9: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['VROC_10'] = df['Volume'].pct_change(10)
/tmp/ipykernel_892722/2545850965.py:10: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['VROC_20'] = df['Volume'].pct_change(20)


Processed and saved: featured_data/ML4T-123.csv
Processed and saved: featured_data/CCMO.csv
Processed and saved: featured_data/ANF.csv
Processed and saved: featured_data/PRU.csv
Processed and saved: featured_data/SVU.csv
Processed and saved: featured_data/WYE.csv
Processed and saved: featured_data/AZO.csv
Processed and saved: featured_data/GIS.csv
Processed and saved: featured_data/ML4T-159.csv
Processed and saved: featured_data/ML4T-010.csv
Processed and saved: featured_data/FMC.csv
Processed and saved: featured_data/RHT.csv
Processed and saved: featured_data/D.csv
Processed and saved: featured_data/CRM.csv
Processed and saved: featured_data/FFIV.csv
Processed and saved: featured_data/HD.csv
Processed and saved: featured_data/ML4T-241.csv
Processed and saved: featured_data/INTC.csv
Processed and saved: featured_data/ML4T-170.csv
Processed and saved: featured_data/SPG.csv
Processed and saved: featured_data/ML4T-195.csv
Processed and saved: featured_data/ML4T-082.csv
Processed and saved

/tmp/ipykernel_892722/2545850965.py:9: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['VROC_10'] = df['Volume'].pct_change(10)
/tmp/ipykernel_892722/2545850965.py:10: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['VROC_20'] = df['Volume'].pct_change(20)


Processed and saved: featured_data/GPS.csv
Processed and saved: featured_data/ML4T-268.csv
Processed and saved: featured_data/AMP.csv
Processed and saved: featured_data/SYY.csv
Processed and saved: featured_data/ML4T-095.csv
Processed and saved: featured_data/ML4T-088.csv
Processed and saved: featured_data/FMCC.csv
Processed and saved: featured_data/WAT.csv
Processed and saved: featured_data/UNM.csv
Processed and saved: featured_data/SJM.csv
Processed and saved: featured_data/ML4T-183.csv
Processed and saved: featured_data/NWSA.csv
Processed and saved: featured_data/TXN.csv
Processed and saved: featured_data/PH.csv
Processed and saved: featured_data/ML4T-322.csv
Processed and saved: featured_data/ML4T-056.csv
Processed and saved: featured_data/ML4T-150.csv
Processed and saved: featured_data/HOT.csv
Processed and saved: featured_data/PXD.csv
Processed and saved: featured_data/LUV.csv
Processed and saved: featured_data/ML4T-246.csv
Processed and saved: featured_data/PDCO.csv
Processed an